# Running simulations in a jupyter notebook 
## Prerequisites
1. A conda environment (conda create --name **myenv** )
2. Update conda sources to include conda forge and bioconda
```
conda config --add channels defaults 
conda config --add channels bioconda 
conda config --add channels conda-forge
```
3. Install all the packages as 
```
conda install -c bioconda gromacs
conda install nglview
conda install py3Dmol
conda install matplotlib
conda install numpy
conda install pytraj
conda install MDAnalysis
conda install jupyter
```
4. Open the jupyter notebook in your conda environment, quite simple in VS Code but also straight forward in conda from terminal.

## Loading all the modules

In [ ]:
import os
import nglview as nv
!jupyter-nbextension enable nglview --py --sys-prefix # Needed to ensure you can visualize nglview objects in the notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import pytraj as pt
import numpy as np
import gromacs
import gromacs.formats
!export GMXLIB=/opt/anaconda3/pkgs/gromacs-2022-nompi_haa54825_0/share/gromacs/top/ # Configure your environmental variable to the directory where all the topologies are stored
import py3Dmol
import MDAnalysis as mda

### Calling GROMACS from terminal

### Making the topology

In [ ]:
!printf '15' | gmx pdb2gmx -f 1aki -o 1_aki_proc -water spce

### Creating the simulation box

In [ ]:
!gmx editconf -f 1aki_proc -c -d 0.5 -bt triclinic -o box

### Solvating the box

In [ ]:
!gmx solvate -cp box -cs spc216 -p topol -o solv

### Integrated GROMACS

In [ ]:
gromacs.solvate(cp='box', cs='spc216', p='topol', o='solv')

## Adding ions

In [ ]:
!gmx grompp -f ions -c solv -p topol -o ions

In [ ]:
!printf '13' | gmx genion -s ions -p topol -neutral -pname NA -nname CL -o solv_ions

## Energy minimization

In [ ]:
!gmx grompp -f em -c solv_ions -p topol -o em
!gmx mdrun -v -deffnm em

## Plotting properties

In [ ]:
!printf "10 0" | gmx energy -f em -o potential
potential = np.genfromtxt([i for i in open('potential.xvg').read().splitlines()
    if not i.startswith(('#','@'))])
plt.plot(*potential.T)
plt.xlabel('step')
plt.ylabel('potential')

## NVT Equilibration

In [ ]:
!gmx grompp -f nvt -c em -p topol -o nvt
!gmx mdrun -v -deffnm nvt

### Plotting pressure evolution during the NVT

In [ ]:
!printf '17' | gmx energy -f nvt -o pressure
pressure = np.genfromtxt([i for i in open('pressure.xvg').read().splitlines()
    if not i.startswith(('#','@'))])
plt.plot(*pressure.T)
plt.xlabel('step')
plt.ylabel('pressure')

## NPT Equilibration

In [ ]:
!gmx grompp -f npt -c nvt -p topol -o npt
!gmx mdrun -v -deffnm npt

## MD Production

In [ ]:
!gmx grompp -f md -c npt -p topol -o md
!gmx mdrun -v -deffnm md

## Visualization

### With py3Dmol

In [ ]:
aki = py3Dmol.view('md.gro')
aki.setStyle('stick')
aki.insert('md.xtc')
aki.show

### With NGLview

In [ ]:
import nglview as nv
import mdtraj as md
traj = md.load('md.xtc',top='md.gro')
t = nv.MDTrajTrajectory(traj)
w = nv.NGLWidget(t)
w

### With MDAnalysis
Allows in a very simple way visualize trajectories

In [ ]:
u = mda.Universe('md.gro','md.xtc')
w = nv.show_mdanalysis(u)
w